In [1]:
import math
from scipy.stats import norm
import pandas as pd
import numpy as np

In [2]:
#Set display precision for float values to 6 decimal places
pd.set_option('display.float_format', '{:.6f}'.format)

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
def linear_congruential_generator(N):
    """Generates uniform random samples on [0,1]."""
    
    ## Parameters for the Linear Congruential Generator.
    a = 39373
    c = 0
    k = 2**31 - 1

    samples = np.zeros(N)
    xi = 1

    for i in range(N):
        xi = (a * xi + c) % k
        ui = xi / k
        samples[i] = ui
    
    return samples


In [5]:
#2.3 Box-Muller
def box_muller(u1,u2):
    u1 = 2 * u1 - 1
    u2 = 2 * u2 - 1
    x = u1**2 + u2**2
    if x > 1:
        return None
    y = np.sqrt(-2*math.log(x)/x)
    z1 = u1 * y
    z2 = u2 * y
    return (z1,z2)

In [6]:
def generate_normal_samples(uniSamples):
    samples = []
    N = 10000*(2**9)
    i = 0
    while len(samples) < N:
        Z = box_muller(uniSamples[i] , uniSamples[i+1])
        if Z:
            samples.append(Z[0])
            samples.append(Z[1])
        i += 2
    return samples

In [7]:
uniformSamples = linear_congruential_generator(10000*(2**14))
normalSamplesAll = generate_normal_samples(uniformSamples)


In [8]:
len(uniformSamples)

163840000

In [9]:
len(normalSamplesAll)

5120000

In [10]:
def blackScholesVanilla(K , T , S , v , q , r , CP ):
    phi = 1 if CP == 'C' else -1
    dp = (1/(v*math.sqrt(T)))*(math.log(S/K) + ((r - q + (v**2)/2)*T) )
    dm = dp - (v*math.sqrt(T))
    vBS = phi*((norm.cdf(phi*dp)*(S*math.exp(-q*T))) - ((norm.cdf(phi*dm))*(K*math.exp(-r*T))))
    return vBS

In [11]:
T = 0.75
CP = 'P'
K = 54
q = 0
S0 = 56
v = 0.27
r = 0.02


In [12]:
V_BS = blackScholesVanilla(K , T , S0 , v , q , r , CP )
print(V_BS)

3.801071883826836


In [ ]:
'''
Z = np.array(normalSamples)
S = S0*np.exp((r-((v**2)/2)) + v*(math.sqrt(T))*Z)
Zero = np.zeros(len(S)) 
V = np.exp(-r*T)*np.maximum(K - S, Zero )

S_n = np.mean(S)
V_n = np.mean(V)

S_bar = S - S_n
V_bar = V - V_n

b = (np.dot(S_bar , V_bar))/(np.dot(S_bar , S_bar))
W = V - b*(S - (math.exp(r*T)*S0))

V_CV_n = W_n = np.mean(W)
'''

In [14]:
#Control Variates
k = list(range(10))
for _k in k:
    n = 10000*(2**_k)
    normalSamples = normalSamplesAll[:n]
    Z = np.array(normalSamples)
    S = S0*np.exp(((r-((v**2)/2))*T) + (v*(math.sqrt(T))*Z))
    Zero = np.zeros(len(S)) 
    V = np.exp(-r*T)*np.maximum(K - S, Zero )

    S_n = np.mean(S)
    V_n = np.mean(V)

    S_bar = S - S_n
    V_bar = V - V_n

    b = (np.dot(S_bar , V_bar))/(np.dot(S_bar , S_bar))
    W = V - b*(S - (math.exp(r*T)*S0))

    V_CV_n = W_n = np.mean(W)
    
    print( n , V_CV_n , abs(V_BS - V_CV_n))

10000 3.762976882118533 0.038095001708303045
20000 3.760174727915477 0.04089715591135912
40000 3.798584417438944 0.002487466387892212
80000 3.805737820114191 0.004665936287354899
160000 3.811181961648319 0.010110077821483099
320000 3.805990838805181 0.004918954978344825
640000 3.804925924377765 0.003854040550928861
1280000 3.8029039866601155 0.001832102833279503
2560000 3.801071882788615 1.0382210646753265e-09
5120000 3.800667107236537 0.0004047765902992495


In [ ]:
len(Z)

In [15]:
#Antithetic Variables
for _k in k:
    n = 10000*(2**_k)
    normalSamples = normalSamplesAll[:n]
    Z = np.array(normalSamples)
    
    Zero = np.zeros(len(Z)) 
    
    Z1 = Z
    Z2 = -1*Z
    
    S1 = S0*np.exp((r-((v**2)/2)) + (v*(math.sqrt(T))*Z1))
    S2 = S0*np.exp(((r-((v**2)/2))*T) + (v*(math.sqrt(T))*Z2))
    
    V1 = np.exp(-r*T)*np.maximum(K - S1, Zero )
    V2 = np.exp(-r*T)*np.maximum(K - S2, Zero )
    
    V = (V1 + V2)/2
    
    V_AV_n = np.mean(V)
    
    print( n , V_AV_n , abs(V_BS - V_AV_n))

10000 3.808513099604818 0.00744121577798218
20000 3.807996772512087 0.006924888685250874
40000 3.83988486465195 0.038812980825114085
80000 3.846151235558733 0.04507935173189681
160000 3.8508171151850608 0.04974523135822473
320000 3.847777442458947 0.04670555863211101
640000 3.847391423336874 0.04631953951003798
1280000 3.845446657931191 0.04437477410435475
2560000 3.843555822274267 0.04248393844743115
5120000 3.843176627732499 0.04210474390566299


In [16]:
# Moment Matching
for _k in k:
    n = 10000*(2**_k)
    normalSamples = normalSamplesAll[:n]
    Z = np.array(normalSamples)
    
    Zero = np.zeros(len(Z)) 
    
    S = S0*np.exp(((r-((v**2)/2))*T) + (v*(math.sqrt(T))*Z))
    S_n = np.mean(S)
    
    S_tilda = S*((math.exp(r*T)*S0)/(S_n))
    
    V_tilda = np.exp(-r*T)*np.maximum(K - S_tilda, Zero )
    
    V_MM_n = np.mean(V_tilda)
    
    print(n , V_MM_n , abs(V_BS - V_MM_n))
    

10000 3.758879248746038 0.04219263508079818
20000 3.7584481485562944 0.04262373527054164
40000 3.797759051858886 0.0033128319679498475
80000 3.805040627815494 0.0039687439886577636
160000 3.8100138105616193 0.008941926734783223
320000 3.806129769131595 0.0050578853047591465
640000 3.8053177682133223 0.0042458843864863205
1280000 3.8030411835162137 0.001969299689377646
2560000 3.8009817806465915 9.010318024449404e-05
5120000 3.800750041218082 0.00032184260875389725


In [17]:
# Simultaneous Moment Matching and Control Variates
for _k in k:
    n = 10000*(2**_k)
    normalSamples = normalSamplesAll[:n]
    Z = np.array(normalSamples)
    
    Zero = np.zeros(len(Z)) 
    
    S = S0*np.exp(((r-((v**2)/2))*T) + (v*(math.sqrt(T))*Z))
    S_n = np.mean(S)
    
    S_tilda = S*((math.exp(r*T)*S0)/(S_n))
    
    V_tilda = np.exp(-r*T)*np.maximum(K - S_tilda, Zero )
    
    S_n = np.mean(S_tilda)
    V_n = np.mean(V_tilda)
    
    S_bar = S_tilda - S_n
    V_bar = V_tilda - V_n

    b = (np.dot(S_bar , V_bar))/(np.dot(S_bar , S_bar))
    W = V_tilda - b*(S - (math.exp(r*T)*S0))

    V_CV_MM_n = np.mean(W)
    print(n , V_CV_MM_n , abs(V_BS - V_CV_MM_n))

10000 3.7275266513559333 0.07354523247090272
20000 3.7457343534426655 0.05533753038417055
40000 3.7918143004676823 0.00925758335915372
80000 3.7998771309924892 0.0011947528343467795
160000 3.8009987406400114 7.314318682460907e-05
320000 3.8071930789511423 0.006121195124306311
640000 3.8083266333965593 0.007254749569723273
1280000 3.8040770605154184 0.003005176688582356
2560000 3.800305630118145 0.0007662537086909893
5120000 3.801372916610829 0.00030103278399318256


In [20]:
T = 0.5

r = 0.025

S10 = 26
S20 = 29

v1 = 0.31
v2 = 0.21

K = 50
rho = 0.3

In [22]:
#Monte Carlo Pricing for Basket Options
k = list(range(9))
for _k in k:
    n = 10000*(2**_k)
    normalSamples = normalSamplesAll[:2*n]
    
    Z1 = np.array(normalSamples)[::2]
    Z2 = np.array(normalSamples)[1::2]
    
    Zero = np.zeros(len(Z1)) 
    
    S1 = S10*np.exp(((r-((v1**2)/2))*T) + (v1*(math.sqrt(T))*Z1))
    S2 = S20*np.exp(((r-((v2**2)/2))*T) + (v2*(math.sqrt(T))*(rho*Z1 + math.sqrt(1-(rho**2))*Z2)))
    
    V = math.exp(-r*T)*np.maximum(S1 + S2 - K , Zero)
    
    V_n = np.mean(V)
    
    print( V_n)


6.635690069726578
6.654352366186453
6.640069192179972
6.641466472571224
6.662836155284208
6.6597729706860775
6.6548515760518345
6.649923570630976
6.652679426499821
